# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv('../output_data/City_list.csv')
city_df.head()


,Unnamed: 0,City_name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,Caborca,30.5838,-111.1010,85.89,8,5,11.74,MX,1619300631
1,1,Iqaluit,63.7506,-68.5145,32.00,86,75,3.44,CA,1619300793
2,2,Busselton,-33.6500,115.3333,51.01,90,0,1.88,AU,1619300634
3,3,Kibondo,-3.5864,30.7203,60.53,95,100,2.08,TZ,1619300794
4,4,Amahai,-3.3333,128.9167,80.01,93,100,3.00,ID,1619300637


In [4]:
city_clean = city_df.drop('Unnamed: 0', 'columns')
city_clean

,City_name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Caborca,30.5838,-111.1010,85.89,8,5,11.74,MX,1619300631
1,Iqaluit,63.7506,-68.5145,32.00,86,75,3.44,CA,1619300793
2,Busselton,-33.6500,115.3333,51.01,90,0,1.88,AU,1619300634
3,Kibondo,-3.5864,30.7203,60.53,95,100,2.08,TZ,1619300794
4,Amahai,-3.3333,128.9167,80.01,93,100,3.00,ID,1619300637
...,...,...,...,...,...,...,...,...,...
567,Kendrapara,20.5042,86.4160,80.60,94,40,4.61,IN,1619301127
568,Bonga,7.2833,36.2333,60.33,98,92,1.77,ET,1619301127
569,Boende,-0.2167,20.8667,70.90,95,64,3.29,CD,1619301128
570,Leninsk,48.7119,45.1994,41.00,87,0,6.71,RU,1619301128


In [5]:
city_clean.head()

,City_name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Caborca,30.5838,-111.1010,85.89,8,5,11.74,MX,1619300631
1,Iqaluit,63.7506,-68.5145,32.00,86,75,3.44,CA,1619300793
2,Busselton,-33.6500,115.3333,51.01,90,0,1.88,AU,1619300634
3,Kibondo,-3.5864,30.7203,60.53,95,100,2.08,TZ,1619300794
4,Amahai,-3.3333,128.9167,80.01,93,100,3.00,ID,1619300637


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
humidity = city_clean["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_clean[["Lat", "Lng"]]

In [9]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
narrowed_city_df = city_clean.loc[(city_clean["Max_temp"] > 70) & (city_clean["Max_temp"] < 80) & (city_clean["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City_name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Santa Luzia,-19.7697,-43.8514,78.80,41,0,3.44,BR,1619300879
1,Buin,-33.7333,-70.7500,73.40,31,0,4.61,CL,1619300962
2,Ṭikāpur,28.5000,81.1333,73.62,33,0,5.06,NP,1619300975
3,Beira,-19.8436,34.8389,71.60,88,0,8.66,MZ,1619301035
4,São João da Barra,-21.6403,-41.0511,77.00,73,0,7.23,BR,1619301089


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotellist = []
for each_hotel in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[each_hotel]['Lat']
    lng = narrowed_city_df.loc[each_hotel]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head(50)

,City_name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
0,Santa Luzia,-19.7697,-43.8514,78.80,41,0,3.44,BR,1619300879,Santa Luzia
1,Buin,-33.7333,-70.7500,73.40,31,0,4.61,CL,1619300962,San Bernardo
2,Ṭikāpur,28.5000,81.1333,73.62,33,0,5.06,NP,1619300975,Tikapur
3,Beira,-19.8436,34.8389,71.60,88,0,8.66,MZ,1619301035,Beira
4,São João da Barra,-21.6403,-41.0511,77.00,73,0,7.23,BR,1619301089,Grussaí
5,Oussouye,12.4850,-16.5469,77.00,88,0,4.61,SN,1619301091,Oussouye
6,Assiut,27.1810,31.1837,71.60,49,0,9.22,EG,1619301098,Asyut
7,Karratha,-20.7377,116.8463,77.00,34,0,7.83,AU,1619301105,Karratha
8,Valentín Gómez Farías,28.2167,-106.5833,75.99,6,0,1.01,MX,1619301122,Santa Cruz de Mayo


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City_name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure
